# Create RIV for NBr44

SFR from NBr43 doesn't cover nearly all of detailed RIV elements. We'll make a new RIV IDF for NBr44 containing only the RIV elements that aren't covered by SFR.

In [1]:
import imod

In [2]:
import WS_Mdl.utils as U
import WS_Mdl.utils_imod as UIM
import WS_Mdl.calcs as C
import WS_Mdl.geo as G

In [3]:
import importlib as IL
IL.reload(U)
IL.reload(UIM)
IL.reload(G)

<module 'WS_Mdl.geo' from 'G:\\code\\WS_Mdl\\geo.py'>

In [4]:
import os
from os import listdir as LD, makedirs as MDs
from os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE
import shutil as sh
import pandas as pd
from datetime import datetime as DT
import matplotlib.pyplot as plt
from pathlib import Path
import re
import xarray as xr

In [5]:
# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True)
# import contextlib

## Options

In [6]:
U.set_verbose(False)

In [7]:
MdlN = 'NBr44'

# Load paths and variables from PRJ & INI
d_Pa = U.get_MdlN_Pa(MdlN)
Pa_PRJ = d_Pa['PRJ']
Dir_PRJ = PDN(Pa_PRJ)
d_INI = U.INI_to_d(d_Pa['INI'])
Xmin, Ymin, Xmax, Ymax, cellsize, N_R, N_C = U.Mdl_Dmns_from_INI(d_Pa['INI'])
SP_date_1st, SP_date_last = [DT.strftime(DT.strptime(d_INI[f'{i}'], '%Y%m%d'), '%Y-%m-%d') for i in ['SDATE', 'EDATE']]
dx = dy = float(d_INI['CELLSIZE'])

Could not determine imod version Sim/NBr44 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
Could not determine imod version Sim/NBr44 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.


# Load

In [8]:
# RIV
A = UIM.xr_clip_Mdl_Aa(imod.idf.open( PJ(d_Pa['In'], 'RIV', f'RIV_Cond_DETAILWATERGANGEN_NBr1.IDF') ), MdlN)

Could not determine imod version Sim/NBr44 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
Could not determine imod version Sim/NBr44 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.


In [9]:
SFR = U.SFR_PkgD_to_DF('NBr40')
SFR

,rno,k,i,j,rlen,rwid,rgrd,rtp,rbth,rhk,man,ncon,ustrf,ndv,line_id,X,Y
0,1,5,291,366,32.71004,1.0,0.0001,18.899851,0.1,0.076429,0.037,1,1,0,7711,122237.5,388937.5
1,2,5,290,365,33.563854,1.0,0.0001,18.899851,0.1,0.074485,0.037,2,1,0,7711,122212.5,388962.5
2,3,5,290,364,3.111411,1.0,0.0001,18.899851,0.1,0.160699,0.037,2,1,0,7711,122187.5,388962.5
3,4,5,289,364,30.346876,1.0,0.0001,18.899851,0.1,0.065905,0.037,2,1,0,7711,122187.5,388987.5
4,5,5,289,363,7.194803,1.0,0.0001,18.899851,0.1,0.138989,0.037,2,1,0,7711,122162.5,388987.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23888,23889,3,19,72,25.761385,1.0,0.0001,4.785,0.1,0.097044,0.037,2,1,0,1575,114887.5,395737.5
23889,23890,3,19,71,20.574833,1.0,0.0001,4.785,0.1,0.072905,0.037,2,1,0,1575,114862.5,395737.5
23890,23891,3,20,71,5.187648,1.0,0.0001,4.785,0.1,0.67468,0.037,2,1,0,1575,114862.5,395712.5
23891,23892,3,20,70,25.803545,1.0,0.0001,4.785,0.1,0.116263,0.037,2,1,0,1575,114837.5,395712.5


# Select RIV elements not in SFR and save to new IDF

In [10]:
import geopandas as gpd
from shapely.geometry import Point

In [11]:
# Create points from X and Y in GDF_SFR
# Find X and Y columns
x_col = next((c for c in SFR.columns if c.lower() == 'x'), None)
y_col = next((c for c in SFR.columns if c.lower() == 'y'), None)

In [12]:
if x_col and y_col:
    # Create GeoDataFrame
    geometry = [Point(xy) for xy in zip(SFR[x_col], SFR[y_col])]
    gdf_points = gpd.GeoDataFrame(SFR, geometry=geometry)

    # Rasterize points to create a mask on A's grid
    # We create a dummy column to rasterize
    gdf_points['present'] = 1
    # imod.prepare.rasterize creates a grid with values where points are
    mask = imod.prepare.rasterize(gdf_points, like=A, column='present')

    # Limit A to items where there is NO point (mask is NaN)
    A1 = A.where(mask.isnull())
    
    print("A has been filtered to exclude locations present in SFR.")
else:
    print("Could not find 'x' and 'y' columns in SFR.")

A has been filtered to exclude locations present in SFR.


In [13]:
imod.idf.save(PJ(d_Pa['In'], f'RIV/{MdlN}', f'RIV_Cond_Detailwatergangen_{MdlN}.IDF'), A1)

# Repeat for Stage
(Although it shouldn't matter)

In [14]:
C = UIM.xr_clip_Mdl_Aa(imod.idf.open( PJ(d_Pa['In'], 'RIV', f'RIV_Stg_DETAILWATERGANGEN_NBr1.IDF') ), MdlN)

Could not determine imod version Sim/NBr44 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.
Could not determine imod version Sim/NBr44 folder doesn't exist, or it's structure has been modified.
 Proceeding with the assumption that it's imod_python.


In [15]:
if x_col and y_col:
    # Create GeoDataFrame
    geometry = [Point(xy) for xy in zip(SFR[x_col], SFR[y_col])]
    gdf_points = gpd.GeoDataFrame(SFR, geometry=geometry)

    # Rasterize points to create a mask on A's grid
    # We create a dummy column to rasterize
    gdf_points['present'] = 1
    # imod.prepare.rasterize creates a grid with values where points are
    mask = imod.prepare.rasterize(gdf_points, like=C, column='present')

    # Limit C to items where there is NO point (mask is NaN)
    C1 = C.where(mask.isnull())
    
    print("C has been filtered to exclude locations present in SFR.")
else:
    print("Could not find 'x' and 'y' columns in SFR.")

C has been filtered to exclude locations present in SFR.


In [16]:
imod.idf.save(PJ(d_Pa['In'], f'RIV/{MdlN}', f'RIV_Stg_Detailwatergangen_{MdlN}.IDF'), C1)